In [1]:
import pandas as pd
import utils
import numpy as np
import warnings
import random
from tqdm import tqdm
import random
from copairs.map import average_precision, mean_average_precision
import logging

logging.basicConfig(format="%(levelname)s:%(asctime)s:%(name)s:%(message)s")
logging.getLogger("copairs").setLevel(logging.INFO)

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
operations = "wellpos_var_mad_int_featselect_harmony_PCA_corrected"
batch_size = 20000
null_size = 20000
fdr = 0.1

### Prepare the data

#### Read the ORF parquet file

In [3]:
crispr_df = pd.read_parquet(
    f"../profiles/profiles_{operations}.parquet"
)
crispr_df.shape

(51185, 309)

#### Remove featues with `nan` values.
These need to be removed as the `nan` values will cause the mean average precision calculation to fail.

In [4]:
crispr_df = utils.remove_nan_features(crispr_df)
crispr_df.shape

Removed nan features: []


(51185, 309)

#### Remove reagents with only one replicate

In [5]:
reagents_with_one_replicate = (
    crispr_df.Metadata_JCP2022.value_counts()[
        crispr_df.Metadata_JCP2022.value_counts() == 1
    ]
    .reset_index()["index"]
    .tolist()
)

crispr_df = crispr_df[~crispr_df.Metadata_JCP2022.isin(reagents_with_one_replicate)]
crispr_df.shape

(51182, 309)

### Calculate mAP for each ORF perturbation

In [6]:
# Adding a new column for negative control
crispr_df["Metadata_negcon"] = np.where(crispr_df["Metadata_pert_type"] == "negcon", 1, 0)

In [7]:
pos_sameby = ["Metadata_JCP2022"]
pos_diffby = []
neg_sameby = ["Metadata_Plate"]
neg_diffby = ["Metadata_negcon"]

In [8]:
metadata_df = utils.get_metadata(crispr_df)
feature_df = utils.get_featuredata(crispr_df)
feature_values = feature_df.values

In [9]:
result = average_precision(
    metadata_df, feature_values, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=batch_size
)

INFO:2024-04-04 15:39:51,160:copairs:Indexing metadata...


INFO:2024-04-04 15:39:51,250:copairs:Finding positive pairs...
INFO:2024-04-04 15:40:01,358:copairs:Finding negative pairs...
INFO:2024-04-04 15:40:02,587:copairs:Computing positive similarities...


  0%|          | 0/761 [00:00<?, ?it/s]

INFO:2024-04-04 15:40:24,576:copairs:Computing negative similarities...


  0%|          | 0/60 [00:00<?, ?it/s]

INFO:2024-04-04 15:40:26,346:copairs:Building rank lists...
INFO:2024-04-04 15:40:54,096:copairs:Computing average precision...
INFO:2024-04-04 15:40:55,673:copairs:Creating result DataFrame...
INFO:2024-04-04 15:40:55,682:copairs:Finished.


In [10]:
agg_result = (
    mean_average_precision(result, pos_sameby, null_size=null_size, threshold=fdr, seed=12527)
    .rename(columns={'average_precision': 'mean_average_precision'})
)

INFO:2024-04-04 15:40:55,854:copairs:Computing null_dist...


  0%|          | 0/128 [00:00<?, ?it/s]

INFO:2024-04-04 15:40:55,968:copairs:Computing p-values...


  0%|          | 0/7974 [00:00<?, ?it/s]

In [11]:
agg_result.to_csv(f"output/phenotypic-activity-{operations}.csv.gz", index=False)